In [1]:
include("Elastic.jl")
using .Elastic
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
using LsqFit

In [17]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 3.61

# 定义铜的FCC晶胞的基矢量
lattice_vectors = collect((Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))')

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,2,2,2)
fcell=filtercell(cpcell)
fig=visualize_unitcell_atoms(fcell)

In [18]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end

ct=5.0
interaction = Interaction(lj, Flj, ct, 0.1)

x=1:0.001:ct
y=interaction.cutenergy.(x)
lines(x,y)


In [19]:
fcell.atoms[1].position+=[0.1,0.1,0.0]
println(cell_energy0(fcell,interaction))
println(force_tensor(fcell,interaction))

-25.20211445242988
[-3.213640279087113e-5 -7.189189836123064e-6 -1.0403382112119576e-5; -7.189189836122963e-6 -3.213640279087112e-5 -1.0403382112119585e-5; 0.0 0.0 1.0401706755968065e-5]


In [20]:
pressure_int(fcell,interaction)

-0.01668556035845475999003259044983383118515192260218942801419455664115905741690972

In [21]:
cell_forceij(fcell,interaction,1,2)

3-element Vector{Float64}:
 -0.00027726189083792876
 -0.00027726189083792876
  0.0027726189083792847

In [15]:
cell_forcei(fcell,interaction,1)

3-element Vector{Float64}:
 0.15972454372988132
 0.15972454372988132
 0.0

In [22]:
dUdV_default(fcell,interaction)

0.01655591058061406357725382283984816721849782121552313010159690039115905741690972

In [13]:
inicell=deepcopy(fcell)
dV::BigFloat=BigFloat("1e-9")
    natom=length(inicell.atoms)
    V0::BigFloat=inicell.Volume
    dV0::BigFloat=inicell.Volume*dV
    V1=V0+dV0
    V2=V0-dV0
    ltv=inicell.lattice_vectors
    ltv1=ltv*(V1/V0)^(1/3)
    ltv2=ltv*(V2/V0)^(1/3)
    cp=inicell.copy

    dcell=deepcopy(inicell)
    dcell.lattice_vectors=ltv1
    for i in 1:natom
        for j in 1:3
            ri=inv((ltv1))*ltv*dcell.atoms[i].position
            println("i=$i,ri=$ri")
            dcell.atoms[i].position[j]=mod(ri[j]+cp[j],2*cp[j])-cp[j]
        end
        # dcell.atoms[i].position=(ltv1)*ltv*dcell.atoms[i].position
    end
    energy1=cell_energy0(dcell,interaction)
    fig=visualize_unitcell_atoms0(dcell,iftext=true)
    display(fig)

i=1,ri=BigFloat[-1.899999999366666667088888923717556244599894811783788354295569059727292089643516, -1.899999999366666667088888923717556244599894811783788354295569059727292089643516, -1.999999999333333333777777814439532889052520854509250899258493747081360094361601]
i=1,ri=BigFloat[-1.899999998733333281564431752731435020226873638414716299613725256142417187838742, -1.899999999366666667088888923717556244599894811783788354295569059727292089643516, -1.999999999333333333777777814439532889052520854509250899258493747081360094361601]
i=1,ri=BigFloat[-1.899999998733333281564431752731435020226873638414716299613725256142417187838742, -1.899999998733333281564431752731435020226873638414716299613725256142417187838742, -1.999999999333333333777777814439532889052520854509250899258493747081360094361601]
i=2,ri=BigFloat[-1.999999999333333333777777814439532889052520854509250899258493747081360094361601, -1.999999999333333333777777814439532889052520854509250899258493747081360094361601, -0.99999999966666666688

GLMakie.Screen(...)

In [12]:
ltv

3×3 Matrix{Float64}:
 10.0   0.0   0.0
  0.0  10.0   0.0
  0.0   0.0  10.0

In [5]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.02:2.5
en=0
fn=0
fl=Vector{Float64}([])
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,3,3,3)
    en=cell_energy0(cpcell,interaction,ifnormalize=true)
    push!(el,en)
    # fn=sum((cell_energy(cpcell,interaction)).^2)
    # push!(fl,fn)
    
end


In [6]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
vl=cl.^3*(cpcell.copy[1]*cpcell.copy[2]*cpcell.copy[3]);
# lines!(ax,cl,fl)
lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig

In [4]:
# 定义 Birch-Murnaghan 方程的函数
function birch_murnaghan(V, p)
    V0, B0, B0_prime, E0 = p
    eta = (V0 ./ V).^(2/3)  # 使用广播运算符 ./
    E = E0 .+ (9 * V0 * B0 / 16) .* ((eta .- 1).^3 .* B0_prime .+ (eta .- 1).^2 .* (6 .- 4 .* eta))
    return E
end

function BMfit(cl,el,p0::Vector{Float64}=[1200.0, -10.0, 4.0, -1200.0])
    fit = curve_fit(birch_murnaghan, vl, el, p0)

    # 拟合结果
    fitted_params = fit.param
    min_lattice_constant = (fitted_params[1]/cpcell.copy[1]/cpcell.copy[2]/cpcell.copy[3])^(1/3)
    println("Fitted parameters:")
    println("V0 = ", fitted_params[1])
    println("B0 = ", fitted_params[2])
    println("B0' = ", fitted_params[3])
    println("E0 = ", fitted_params[4])
    println("latticeconstant = ",min_lattice_constant )
    fig = Figure()
    fitfunc(x)=birch_murnaghan(x, fitted_params)
    # 创建一个 Axis，并设置坐标范围
    ax = Axis(fig[1, 1],xlabel="Volume", ylabel="Energy", title="Birch-Murnaghan EOS Fit"
              )
    x=LinRange(minimum(vl),maximum(vl),1000)
    scatter!(ax,vl,el,color=:red,label="Data")
    lines!(ax,x,fitfunc.(x),color=:blue,label="Fit")
    axislegend(ax, position=:rt)
    fig
    return min_lattice_constant,fig
end

BMfit (generic function with 2 methods)

In [8]:
_,fig=BMfit(vl,el)
fig

Fitted parameters:
V0 = 326.7246123157035
B0 = -5.046009356822658
B0' = 2.820280876351377
E0 = -141.49284306089007
latticeconstant = 2.295828074709017


In [3]:
function cell_forceij(cell::UnitCell, interaction::Interaction, i::Int, j::Int)
    cutoff = interaction.cutoff
    atomi = cell.atoms[i]
    atomj = cell.atoms[j]
    bd=abs.(atomj.bound)
    cp = cell.copy
    cni = atomi.cn
    rij=atomj.position-atomi.position
    for k in 1:3
        rijk=rij[k]
        cpk=cp[k]
        # if bd[k]==1
        #     continue
        # end
        if rijk>cpk/2
                rij[k]=rijk-cpk
        elseif rijk<-cpk/2
                rij[k]=rijk+cpk
        end
    end
    rij=cell.lattice_vectors*rij
    nr=norm(rij)
    if nr>cutoff
        force=zeros(3)
    else
        force=interaction.cutforce(rij)
    end

    if any(isnan,force)
        println("Warning the same atoms of atom $i j=$j lead to the nan force,please check their cn")
    end
    return force
end

function cell_forcei(cell::UnitCell,interaction::Interaction,i::Int)
    forcei=zeros(3)
        for j in 1:length(cell.atoms)
            if i!=j
                forcei+=cell_forceij(cell,interaction,i,j)
            end
        end
    return forcei
end



function force_tensor(cell::UnitCell,interaction::Interaction)
    tensor=zeros(3,3)
    v=cell.Volume
    for a in 1:3
        for b in 1:3
            for i in 1:length(cell.atoms)
                forcei=cell_forcei(cell,interaction,i)
                atom=cell.atoms[i]
                tensor[a,b]+=forcei[a]*atom.position[b]/2+atom.momentum[a]*atom.momentum[b]/atom.mass
            end
        end
    end
    return tensor./v
end


force_tensor (generic function with 1 method)

In [2]:

###MD
function dUdV_default(r::Vector{Float64},v::Float64)
    return 0.0
end

function pressure_int(cell::UnitCell,interaction::Interaction,dUdV::T=dUdV_default) where T
    v=cell.Volume
    Pint=0.0
    for i in 1:length(cell.atoms)
        atom=cell.atoms[i]
        pm=atom.momentum
        ri=atom.position
        fi=cell_forcei(cell,interaction,i)
        Pint+=dot(pm,pm)/atom.mass+dot(ri,fi)-3*v*dUdV(atom.position,v)   
    end
    return Pint/v/3
    
end


struct Thermostat
    T::Float64  # 目标温度
    Q::Float64  # 热浴质量
    Rt::Float64  # 热浴变量 (friction coefficient)
    Pt::Float64  # 热浴动量
end
struct Barostat
    Pe::Float64  # 目标压力
    V::Float64  # 系统体积
    W::Float64  # 压力浴质量
    Pv::Float64  # 压力浴动量
end


function Hz(z::Vector{Float64},cell::UnitCell,interaction::Interaction,thermostat::Thermostat,barostat::Barostat,dUdV::T=dUdV_default) where T
    kb=1.38064852e-23
    dim=3*length(cell.atoms)+2
    Hz=zeros(dim*2)
    natom=length(cell.atoms)
    # if dim!=length(z)
    #     throw("The dimension of z is not consist with the dimension of the system. z should be natom*3+2")
    # end
    v=cell.Volume
    W=barostat.W
    Q=thermostat.Q
    temp=thermostat.T
    Pe=barostat.Pe
    Pint=pressure_int(cell,interaction,dUdV)
    addp=sum([dot(atom.momentum,atom.momentum)/atom.mass for atom in cell.atoms])
    for i in 1:natom
        atom=cell.atoms[i]
        mi=atom.mass
        Hz[3*i-2]=z[3*i-2+dim]/mi+z[2*dim]*z[3*i-2]/W
        Hz[3*i-1]=z[3*i-1+dim]/mi+z[2*dim]*z[3*i-1]/W
        Hz[3*i]=z[3*i+dim]/mi+z[2*dim]*z[3*i]/W
        fi=cell_forcei(cell,interaction,i)
        Hz[dim+3*i-2]=fi[1]-(1+1/natom)*z[2*dim]/W*z[dim+3*i-2]-z[2*dim-1]*z[dim+3*i-2]/Q
        Hz[dim+3*i-1]=fi[2]-(1+1/natom)*z[2*dim]/W*z[dim+3*i-1]-z[2*dim-1]*z[dim+3*i-1]/Q
        Hz[dim+3*i]=fi[3]-(1+1/natom)*z[2*dim]/W*z[dim+3*i]-z[2*dim-1]*z[dim+3*i]/Q
        Hz[dim]=3*z[dim]*z[2*dim]/W
        Hz[2*dim]=3*z[dim]*(Pint-Pe)+1/natom*addp-z[2*dim-1]*z[2*dim]/Q
        Hz[dim-1]=z[2*dim-1]/Q
        Hz[2*dim-1]=addp+z[2*dim]^2/W-(3*natom+1)*kb*temp
    end
    return Hz
end


function symplectic_matrix(n::Int)
    I = Matrix{Float64}(I, n, n)  # 创建 n x n 的单位矩阵
    O = zeros(Float64, n, n)      # 创建 n x n 的零矩阵
    J = [O I; -I O]               # 使用块矩阵构建辛矩阵
    return J
end

symplectic_matrix (generic function with 1 method)

In [71]:

# 定义 symplectic_matrix 函数
function symplectic_matrix(n::Int)
    I = Matrix{Float64}(I, n, n)  # 创建 n x n 的单位矩阵
    O = zeros(Float64, n, n)      # 创建 n x n 的零矩阵
    J = [O I; -I O]               # 使用块矩阵构建辛矩阵
    return J
end

function RK3_step(z::Vector{Float64},dt::Float64,cell::UnitCell, interaction::Interaction, thermostat::Thermostat, barostat::Barostat, dUdV::T = dUdV_default) where T
    k1=Hz(z,cell,interaction,thermostat,barostat,dUdV)
    k2=Hz(z+dt/2*k1,cell,interaction,thermostat,barostat,dUdV)
    k3=Hz(z-dt*k1+2*dt*k2,cell,interaction,thermostat,barostat,dUdV)
    newz=z+dt/6*(k1+4*k2+k3)
    zmod=-1*ones(length(newz))
    a,b,c=cell.lattice_vectors*cell.copy
    natom=length(cell.atoms)
    for i in 1:natom
        zmod[3*i-2]=mod(newz[3*i-2],a)
        zmod[3*i-1]=mod(newz[3*i-1],b)
        zmod[3*i]=mod(newz[3*i],c)
        zmod[3*natom+3*i]=mod(newz[3*natom+3*i],a)
        zmod[3*natom+3*i+1]=mod(newz[3*natom+3*i+1],b)
        zmod[3*natom+3*i+2]=mod(newz[3*natom+3*i+2],c)
    end
    zmod[3*natom+1]=newz[3*natom+1]
    zmod[3*natom+2]=newz[3*natom+2]
    zmod[2*3*natom+3]=newz[2*3*natom+3]
    zmod[2*3*natom+4]=newz[2*3*natom+4]
return zmod
end
function filtercell(cell::UnitCell)
    a,b,c=cell.lattice_vectors*cell.copy
    natom=length(cell.atoms)
    atoms=Vector{Atom}([])
    for i in 1:natom
        if -1 in cell.atoms[i].bound
            continue
        end
        push!(atoms,cell.atoms[i])
    end
    return UnitCell(cell.lattice_vectors,atoms,cell.copy)
end

filtercell (generic function with 1 method)

In [72]:
# 定义铜的晶格常数（单位：Å）
lattice_constant =1

# 定义铜的FCC晶胞的基矢量
lattice_vectors = collect((Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))')

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,3,3,3)
fcell=filtercell(cpcell)
visualize_unitcell_atoms(fcell)

In [1]:
thermostat = Thermostat(300.0, 10.0, 0.1, 0.0)
barostat=Barostat(15.0,fcell.Volume,10.0,0.0)
rl=[fcell.lattice_vectors*atom.position for atom in fcell.atoms];
pl=[atom.momentum for atom in fcell.atoms];
z=vcat(vcat(rl...),thermostat.Rt,barostat.V,vcat(pl...),thermostat.Pt,barostat.Pv);
z

UndefVarError: UndefVarError: `Thermostat` not defined

In [77]:
RK3_step(z,0.01,fcell,interaction,thermostat,barostat)

652-element Vector{Float64}:
 2.9997313889910746
 2.999731388991075
 2.999731388991075
 2.9997313889910746
 2.999731388991075
 1.0001827996492583
 2.9997313889910746
 2.999731388991075
 2.001171432325292
 2.9997313889910746
 ⋮
 2.9463021128281954
 0.05369788717183207
 0.05369788717179577
 2.9463021128281945
 0.05369788717182269
 0.05369788717179549
 0.05369788717180458
 0.00027153203773085013
 0.9029570211179258

In [78]:
zl=[]
for i in 1:100
    push!(zl,z)
    z=RK3_step(z,0.001,fcell,interaction,thermostat,barostat)
end

In [79]:
pressure_int(fcell,interaction)
zl

100-element Vector{Any}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [2.999997314286165, 2.999997314286165, 2.999997314286165, 2.999997314286165, 2.999997314286165, 1.0000018270486635, 2.999997314286165, 2.999997314286165, 2.000011711238831, 2.999997314286165  …  0.005371403354791034, 2.994628596645208, 0.005371403354794693, 0.005371403354791061, 2.994628596645208, 0.0053714033547937546, 0.005371403354791034, 0.005371403354791943, 2.715320377308487e-7, 0.09025538573035642]
 [0.0030298769237981915, 0.0030298769237981915, 0.0030298769237981915, 0.0030298769237981915, 0.0030298769237981915, 1.003007312778675, 0.0030298769237981915, 0.0030298769237981915, 2.0000468454909974, 0.0030298769237981915  …  0.01074270885102813, 2.9892162998995864, 0.01074270885103545, 0.010742708851028187, 2.989216299899586, 0.010742708851033574, 0.01074270885102813, 0.01074270885102995, 2.1723004153047157e-6, 0.18051321529439096]
 [0.00601648008348214

In [80]:
function z2atoms(z::Vector{Float64},cell::UnitCell)
    natom=Int((length(z)-4)/6)
    rl=z[1:3*natom]
    pl=z[3*natom+3:3*natom+3*natom+3]
    atoms=Vector{Atom}(undef,natom)
    for i in 1:natom
        atom=Atom(rl[3*i-2:3*i],pl[3*i-2:3*i],cell.atoms[i].mass,cell.atoms[i].cn,cell.atoms[i].bound)
        atoms[i]=atom
    end
    return atoms
end

function z2cell(z::Vector{Float64},cell::UnitCell)
    atoms=z2atoms(z,cell)
    newcell=UnitCell(cell.lattice_vectors,atoms,cell.copy)
    return newcell
end

z2cell (generic function with 1 method)

In [94]:
newcell=z2cell(zl[20],fcell)
fig=visualize_unitcell_atoms0(newcell)
fig

In [95]:
pressure_int(newcell,interaction)

-12740.486390678687

In [54]:
# 定义一个颜色映射函数
function color_map(cn)
    colors = [:red, :green, :blue, :yellow, :purple, :orange, :cyan, :magenta]
    return colors[mod(cn - 1, length(colors)) + 1]
end
"""
可视化晶胞原子
"""
function visualize_unitcell_atoms0(cell::UnitCell,markersize=10,veccolor=:blue,linewith=0.1)::Figure
    fig =GLMakie.Figure(size = (800, 600))
    ax = GLMakie.Axis3(fig[1, 1], title = "Visualization of Atoms in the Unit Cell", 
               xlabel = "X", ylabel = "Y", zlabel = "Z")
    M=cell.lattice_vectors
    for atom in cell.atoms
        p=M*atom.position
        cni=atom.cn
        GLMakie.scatter!(ax,p..., color = color_map(cni), markersize = markersize)
    end
    return fig
end

ErrorException: invalid method definition in Main: function Model.visualize_unitcell_atoms0 must be explicitly imported to be extended

In [55]:
using Makie
using FFMPEG

# 假设func(i)是一个生成3D图的函数
function func(i)
    newcell=z2cell(zl[i],lattice_vectors)
    fig=visualize_unitcell_atoms0(newcell)
    return fig
end

function create_animation()
    video_file = "outputatom.mp4"
    n_frames = 1000
    fps = 1
    # 使用record函数生成动画
    record(fig, video_file, 1:n_frames; framerate = fps) do i
        func(i)
    end
end


# 调用函数生成视频
create_animation()


MethodError: MethodError: no method matching z2cell(::Vector{Float64}, ::Matrix{Float64})

Closest candidates are:
  z2cell(::Vector{Float64}, !Matched::UnitCell)
   @ Main c:\Users\ASUS\Desktop\计算物理模拟实验\Elastic constant simulation\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:13
